In [ ]:
import os
import cv2
import numpy as np
from config import *
from data_preprocessor import *
from model import *
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
import gc
import psutil
import json

In [ ]:
X_train, X_text, y_train, y_test = read_data()

In [ ]:
def load_init_params():
    with open("initial_params.json", "r") as file:
        data = json.load(file)
    
    return data

class CuckooSearchOptimization:
    def __init__(self, X_train, y_train, nests, num_nests=3, num_iterations=4, pa=0.25):
        self.X_train = X_train
        self.y_train = y_train
        self.num_nests = num_nests
        self.num_iterations = num_iterations
        self.pa = pa  # discovery probability
        self.param_space = {
            'lr': [0.0001, 0.01],
            'batch_size': [16, 64],
            'filters1': [32, 64],
            'filters2': [32, 64],
            'units1': [32, 64],
            'dropout': [0.1, 0.5],
        }
        # self.nests = [self.random_params() for _ in range(num_nests)]
        self.nests = nests
        print(self.nests)
        self.fitness = np.array([objective_function(p, X_train, y_train) for p in self.nests])

    def random_params(self):
        return {
            'lr': np.random.uniform(*self.param_space['lr']),
            'batch_size': int(np.random.uniform(*self.param_space['batch_size'])),
            'filters1': int(np.random.uniform(*self.param_space['filters1'])),
            'filters2': int(np.random.uniform(*self.param_space['filters2'])),
            'units1': int(np.random.uniform(*self.param_space['units1'])),
            'dropout': np.random.uniform(*self.param_space['dropout']),
        }

    def levy_flight(self, params):
        new_params = params.copy()
        for key in new_params:
            step = np.random.normal(0, 0.1)  # small step
            if key in ['filters1', 'filters2', 'units1', 'batch_size']:
                new_params[key] = int(np.clip(new_params[key] + step * 10, *self.param_space[key]))
            else:
                new_params[key] = np.clip(new_params[key] + step, *self.param_space[key])
        return new_params

    def optimize(self):
        for iter_num in range(self.num_iterations):
            print(f"\nIteration {iter_num + 1}/{self.num_iterations}")
            for i in range(self.num_nests):
                new_params = self.levy_flight(self.nests[i])
                new_fitness = objective_function(new_params, self.X_train, self.y_train)
                if new_fitness > self.fitness[i]:
                    print(f"Nest {i} improved.")
                    self.nests[i] = new_params
                    self.fitness[i] = new_fitness

            # Discovery and randomization
            for i in range(self.num_nests):
                if np.random.rand() < self.pa:
                    self.nests[i] = self.random_params()
                    self.fitness[i] = objective_function(self.nests[i], self.X_train, self.y_train)

            best_iter_idx = np.argmin(self.fitness)
            print(f"Best so far: Accuracy = {-self.fitness[best_iter_idx]:.4f}")

        best_index = np.argmin(self.fitness)
        return self.nests[best_index], -self.fitness[best_index]

# Run Cuckoo Search Optimization
nests = load_init_params()
cso = CuckooSearchOptimization(X_train, y_train, nests)
best_params, best_accuracy = cso.optimize()
print("\n Best Hyperparameters:", best_params)
print("Best Validation Accuracy:", best_accuracy)

[{'lr': 0.008789098700666468, 'batch_size': 22, 'filters1': 52, 'filters2': 48, 'units1': 45, 'dropout': 0.23562910689945527}, {'lr': 0.007011921512008315, 'batch_size': 26, 'filters1': 47, 'filters2': 54, 'units1': 48, 'dropout': 0.39452114202640026}, {'lr': 0.00984578875390895, 'batch_size': 56, 'filters1': 49, 'filters2': 51, 'units1': 52, 'dropout': 0.1523667422741935}]
Evaluated Params: {'lr': 0.008789098700666468, 'batch_size': 22, 'filters1': 52, 'filters2': 48, 'units1': 45, 'dropout': 0.23562910689945527}
Validation Accuracy: 0.2266
Memory usage (MB): 9517.28125
Evaluated Params: {'lr': 0.007011921512008315, 'batch_size': 26, 'filters1': 47, 'filters2': 54, 'units1': 48, 'dropout': 0.39452114202640026}
Validation Accuracy: 0.2266
Memory usage (MB): 9680.75
Evaluated Params: {'lr': 0.00984578875390895, 'batch_size': 56, 'filters1': 49, 'filters2': 51, 'units1': 52, 'dropout': 0.1523667422741935}
Validation Accuracy: 0.2266
Memory usage (MB): 8073.15625

Iteration 1/4
Evaluated 